순환 신경망을 사용해 다음 단어를 예측하는 언어 모델링 - LSTM

In [3]:
from __future__ import print_function
import os

from sklearn.model_selection import train_test_split
import nltk
import numpy as np
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
with open('/content/drive/MyDrive/Colab Notebooks/13주차_과제_김정웅(20175308)/alice_in_wonderland.txt', 'r') as content_file:
    content = content_file.read()

content2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in content]).split())
 
tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

In [6]:
N = 3
quads = list(nltk.ngrams(tokens,N))

newl_app = []
for ln in quads:
    newl = " ".join(ln)        
    newl_app.append(newl)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
# 단어 벡터화
vectorizer = CountVectorizer()

x_trigm = []
y_trigm = []

for l in newl_app:
    x_str = " ".join(l.split()[0:N-1])
    y_str = l.split()[N-1]   
    x_trigm.append(x_str)
    y_trigm.append(y_str)

x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()

In [8]:
dictnry = vectorizer.vocabulary_
rev_dictnry = {v:k for k,v in dictnry.items()}

X = np.array(x_trigm_check)
Y = np.array(y_trigm_check)

Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X, Y,x_trigm, test_size=0.3,random_state=42)

print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape)
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17947, 2559) Y Train shape (17947, 2559)
X Test shape (7692, 2559) Y Test shape (7692, 2559)


In [9]:
from keras.layers import Input,Dense,Dropout
from keras.models import Model

np.random.seed(42)

BATCH_SIZE = 128
NUM_EPOCHS = 100

input_layer = Input(shape = (Xtrain.shape[1],),name="input")
first_layer = Dense(1000,activation='relu',name = "first")(input_layer)
first_dropout = Dropout(0.5,name="firstdout")(first_layer)

second_layer = Dense(800,activation='relu',name="second")(first_dropout)

third_layer = Dense(1000,activation='relu',name="third")(second_layer)
third_dropout = Dropout(0.5,name="thirdout")(third_layer)

fourth_layer = Dense(Ytrain.shape[1],activation='softmax',name = "fourth")(third_dropout)


history = Model(input_layer,fourth_layer)
history.compile(optimizer = "adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [10]:
print (history.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 2559)]            0         
                                                                 
 first (Dense)               (None, 1000)              2560000   
                                                                 
 firstdout (Dropout)         (None, 1000)              0         
                                                                 
 second (Dense)              (None, 800)               800800    
                                                                 
 third (Dense)               (None, 1000)              801000    
                                                                 
 thirdout (Dropout)          (None, 1000)              0         
                                                                 
 fourth (Dense)              (None, 2559)              256155

In [11]:
history.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS, verbose=1,validation_split = 0.2)

Epoch 1/100
113/113 [==============================] - 19s 165ms/step - loss: 6.3500 - accuracy: 0.0553 - val_loss: 6.0907 - val_accuracy: 0.0663
Epoch 2/100
113/113 [==============================] - 19s 172ms/step - loss: 5.9139 - accuracy: 0.0651 - val_loss: 6.0967 - val_accuracy: 0.0699
Epoch 3/100
113/113 [==============================] - 18s 161ms/step - loss: 5.6933 - accuracy: 0.0789 - val_loss: 5.9844 - val_accuracy: 0.0819
Epoch 4/100
113/113 [==============================] - 18s 159ms/step - loss: 5.4426 - accuracy: 0.1044 - val_loss: 5.9695 - val_accuracy: 0.0905
Epoch 5/100
113/113 [==============================] - 18s 161ms/step - loss: 5.1785 - accuracy: 0.1280 - val_loss: 6.0143 - val_accuracy: 0.1022
Epoch 6/100
113/113 [==============================] - 18s 163ms/step - loss: 4.9318 - accuracy: 0.1545 - val_loss: 6.1652 - val_accuracy: 0.1075
Epoch 7/100
113/113 [==============================] - 18s 160ms/step - loss: 4.6955 - accuracy: 0.1766 - val_loss: 6.1897 -

In [12]:
Y_pred = history.predict(Xtest)

In [13]:
print ("Prior bigram words","|Actual","|Predicted","\n")

Prior bigram words |Actual |Predicted 



In [14]:
for i in range(10):
    print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

0 the evening | beautiful | beautiful
1 slipped in | like | the
2 alice swallowing | down | not
3 an encouraging | tone | thing
4 waistcoat pocket | or | and
5 she went | on | on
6 that she | knew | was
7 down on | her | one
8 dormouse went | on | on
9 soup soup | of | there


In [15]:
import random
	
NUM_DISPLAY = 10
for i in random.sample(range(len(xtest_tg)),NUM_DISPLAY):
	print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])],"|",rev_dictnry[np.argmax(Y_pred[i])])

5790 seated on | their | for
1310 to say | than | it
6088 read in | the | her
1532 four thousand | miles | inches
7375 hearts next | came | she
4626 king hastily | said | down
6301 besides she | she | had
6546 appeared she | was | said
5462 he added | turning | looking
502 puzzled he | must | said
